# SETTINGS

In [61]:
############ LIBRARIES

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('dark_background')
%matplotlib inline

import os
import time
import datetime
import random
import multiprocessing
import pickle

import scipy.stats

import gc
gc.enable()

import warnings
warnings.filterwarnings('ignore')

In [62]:
############ HELPER FUNCTIONS

import functions
import importlib
importlib.reload(functions)
from functions import *

In [63]:
############ RANDOMNESS

# seed function
def seed_everything(seed = 23):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    
# set seed
seed = 23
seed_everything(seed)

# IMPORT

In [64]:
# import CSV
df = pd.read_pickle('../input/data.pkl')
print(df.shape)

(1097231, 434)


In [65]:
# target variable
target = 'isFraud'

# FEATURE ENGINEERING

In [66]:
############ E-MAIL DOMAINS

print(df.shape)

emails = {'gmail': 'google', 'att.net': 'att', 'twc.com': 'spectrum', 'scranton.edu': 'other', 'optonline.net': 'other',
          'hotmail.co.uk': 'microsoft', 'comcast.net': 'other', 'yahoo.com.mx': 'yahoo', 'yahoo.fr': 'yahoo',
          'yahoo.es': 'yahoo', 'charter.net': 'spectrum', 'live.com': 'microsoft', 'aim.com': 'aol', 'hotmail.de': 'microsoft',
          'centurylink.net': 'centurylink', 'gmail.com': 'google', 'me.com': 'apple', 'earthlink.net': 'other', 
          'gmx.de': 'other', 'web.de': 'other', 'cfl.rr.com': 'other', 'hotmail.com': 'microsoft', 'protonmail.com': 'other',
          'hotmail.fr': 'microsoft', 'windstream.net': 'other', 'outlook.es': 'microsoft', 'yahoo.co.jp': 'yahoo',
          'yahoo.de': 'yahoo', 'servicios-ta.com': 'other', 'netzero.net': 'other', 'suddenlink.net': 'other',
          'roadrunner.com': 'other', 'sc.rr.com': 'other', 'live.fr': 'microsoft', 'verizon.net': 'yahoo',
          'msn.com': 'microsoft', 'q.com': 'centurylink', 'prodigy.net.mx': 'att', 'frontier.com': 'yahoo',
          'anonymous.com': 'other', 'rocketmail.com': 'yahoo', 'sbcglobal.net': 'att', 'frontiernet.net': 'yahoo',
          'ymail.com': 'yahoo', 'outlook.com': 'microsoft', 'mail.com': 'other', 'bellsouth.net': 'other',
          'embarqmail.com': 'centurylink', 'cableone.net': 'other', 'hotmail.es': 'microsoft', 'mac.com': 'apple',
          'yahoo.co.uk': 'yahoo', 'netzero.com': 'other', 'yahoo.com': 'yahoo', 'live.com.mx': 'microsoft', 'ptd.net': 'other',
          'cox.net': 'other', 'aol.com': 'aol', 'juno.com': 'other', 'icloud.com': 'apple'}
us_emails = ['gmail', 'net', 'edu']

for c in ['P_emaildomain', 'R_emaildomain']:
    df[c + '_bin']    = df[c].map(emails)
    df[c + '_suffix'] = df[c].map(lambda x: str(x).split('.')[-1])
    df[c + '_suffix'] = df[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')
    df[c + '_prefix'] = df[c].apply(lambda x: str(x).split('.')[0])
    

df['P_emaildomain'] = df['P_emaildomain'].fillna('unknown')
df['R_emaildomain'] = df['R_emaildomain'].fillna('unknown')
    
df['email_match'] = np.where((df['P_emaildomain'] == df['R_emaildomain']) & (df['P_emaildomain'] != 'unknown'), 1, 0)
    
del df['P_emaildomain'], df['R_emaildomain']

print(df.shape)

(1097231, 434)
(1097231, 439)


In [67]:
############ DEVICE TYPE

print(df.shape)

df['DeviceInfo']  = df['DeviceInfo'].fillna('unknown_device').str.lower()
df['device_name'] = df['DeviceInfo'].str.split('/', expand = True)[0]

df.loc[df['device_name'].str.contains('SM', na=False),      'device_name'] = 'Samsung'
df.loc[df['device_name'].str.contains('sm', na=False),      'device_name'] = 'Samsung'
df.loc[df['device_name'].str.contains('SAMSUNG', na=False), 'device_name'] = 'Samsung'
df.loc[df['device_name'].str.contains('GT-', na=False),     'device_name'] = 'Samsung'
df.loc[df['device_name'].str.contains('Moto G', na=False),  'device_name'] = 'Motorola'
df.loc[df['device_name'].str.contains('Moto', na=False),    'device_name'] = 'Motorola'
df.loc[df['device_name'].str.contains('moto', na=False),    'device_name'] = 'Motorola'
df.loc[df['device_name'].str.contains('LG-', na=False),     'device_name'] = 'LG'
df.loc[df['device_name'].str.contains('lg-', na=False),     'device_name'] = 'LG'
df.loc[df['device_name'].str.contains('rv:', na=False),     'device_name'] = 'RV'
df.loc[df['device_name'].str.contains('HUAWEI', na=False),  'device_name'] = 'Huawei'
df.loc[df['device_name'].str.contains('huawei', na=False),  'device_name'] = 'Huawei'
df.loc[df['device_name'].str.contains('ALE-', na=False),    'device_name'] = 'Huawei'
df.loc[df['device_name'].str.contains('ale-', na=False),    'device_name'] = 'Huawei'
df.loc[df['device_name'].str.contains('ane-', na=False),    'device_name'] = 'Huawei'
df.loc[df['device_name'].str.contains('cam-', na=False),    'device_name'] = 'Huawei'
df.loc[df['device_name'].str.contains('rne-', na=False),    'device_name'] = 'Huawei'
df.loc[df['device_name'].str.contains('pra-', na=False),    'device_name'] = 'Huawei'
df.loc[df['device_name'].str.contains('-L', na=False),      'device_name'] = 'Huawei'
df.loc[df['device_name'].str.contains('Blade', na=False),   'device_name'] = 'ZTE'
df.loc[df['device_name'].str.contains('BLADE', na=False),   'device_name'] = 'ZTE'
df.loc[df['device_name'].str.contains('blade', na=False),   'device_name'] = 'ZTE'
df.loc[df['device_name'].str.contains('Linux', na=False),   'device_name'] = 'Linux'
df.loc[df['device_name'].str.contains('linux', na=False),   'device_name'] = 'Linux'
df.loc[df['device_name'].str.contains('XT', na=False),      'device_name'] = 'Sony'
df.loc[df['device_name'].str.contains('xt', na=False),      'device_name'] = 'Sony'
df.loc[df['device_name'].str.contains('HTC', na=False),     'device_name'] = 'HTC'
df.loc[df['device_name'].str.contains('htc', na=False),     'device_name'] = 'HTC'
df.loc[df['device_name'].str.contains('ASUS', na=False),    'device_name'] = 'Asus'
df.loc[df['device_name'].str.contains('asus', na=False),    'device_name'] = 'Asus'

df.loc[df.device_name.isin(df.device_name.value_counts()[df.device_name.value_counts() < 200].index), 'device_name'] = "Others"

#del df['DeviceInfo']

print(df.shape)

(1097231, 439)
(1097231, 440)


In [68]:
############ OPERATING SYSTEM

print(df.shape)

df['operating_system'] = np.NaN

df.loc[df['id_30'].str.contains('Windows', na = False), 'operating_system']  = 'Windows'
df.loc[df['id_30'].str.contains('iOS',     na = False), 'operating_system']  = 'iOS'
df.loc[df['id_30'].str.contains('Mac OS',  na = False), 'operating_system']  = 'MacOS'
df.loc[df['id_30'].str.contains('Android', na = False), 'operating_system']  = 'Android'
df.loc[df['id_30'].str.contains('Linux',   na = False), 'operating_system']  = 'Linux'

df.loc[df['operating_system'].isnull(), 'operating_system'] = "Other"
df.loc[df['id_30'].isnull(),            'operating_system'] = "Unknown"

print(df.shape)

(1097231, 440)
(1097231, 441)


In [69]:
############ BROWSER TYPE

print(df.shape)

df['browser_type'] = np.NaN

df.loc[df['id_31'].str.contains('android', na = False),   'browser_type']  = 'Android'
df.loc[df['id_31'].str.contains('chrome', na = False),    'browser_type']  = 'Chrome'
df.loc[df['id_31'].str.contains('opera',     na = False), 'browser_type']  = 'Opera'
df.loc[df['id_31'].str.contains('edge',  na = False),     'browser_type']  = 'Edge'
df.loc[df['id_31'].str.contains('firefox', na = False),   'browser_type']  = 'Firefox'
df.loc[df['id_31'].str.contains('safari', na = False),    'browser_type']  = 'Safari'
df.loc[df['id_31'].str.contains('samsung', na = False),   'browser_type']  = 'Samsung'
df.loc[df['id_31'].str.contains('ie', na = False),        'browser_type']  = 'Internet Explorer'
df.loc[df['id_31'].str.contains('google', na = False),    'browser_type']  = 'Google Search'

df.loc[df['browser_type'].isnull(), 'browser_type'] = "Other"
df.loc[df['id_31'].isnull(),        'browser_type'] = "Unknown"

print(df.shape)

(1097231, 441)
(1097231, 442)


In [70]:
############ BROWSER VERSION

print(df.shape)

df['lastest_browser'] = np.zeros(df.shape[0])

df.loc[df["id_31"] == "samsung browser 7.0",'lastest_browser']=1
df.loc[df["id_31"] == "opera 53.0",'lastest_browser']=1
df.loc[df["id_31"] == "mobile safari 10.0",'lastest_browser']=1
df.loc[df["id_31"] == "google search application 49.0",'lastest_browser']=1
df.loc[df["id_31"] == "firefox 60.0",'lastest_browser']=1
df.loc[df["id_31"] == "edge 17.0",'lastest_browser']=1
df.loc[df["id_31"] == "chrome 69.0",'lastest_browser']=1
df.loc[df["id_31"] == "chrome 67.0 for android",'lastest_browser']=1
df.loc[df["id_31"] == "chrome 63.0 for android",'lastest_browser']=1
df.loc[df["id_31"] == "chrome 63.0 for ios",'lastest_browser']=1
df.loc[df["id_31"] == "chrome 64.0",'lastest_browser']=1
df.loc[df["id_31"] == "chrome 64.0 for android",'lastest_browser']=1
df.loc[df["id_31"] == "chrome 64.0 for ios",'lastest_browser']=1
df.loc[df["id_31"] == "chrome 65.0",'lastest_browser']=1
df.loc[df["id_31"] == "chrome 65.0 for android",'lastest_browser']=1
df.loc[df["id_31"] == "chrome 65.0 for ios",'lastest_browser']=1
df.loc[df["id_31"] == "chrome 66.0",'lastest_browser']=1
df.loc[df["id_31"] == "chrome 66.0 for android",'lastest_browser']=1
df.loc[df["id_31"] == "chrome 66.0 for ios",'lastest_browser']=1

print(df.shape)

(1097231, 442)
(1097231, 443)


In [71]:
############ NUMBER OF MATCHES

print(df.shape)

for var in ['M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9']:
    df[var].fillna('N', inplace = True)

df['M'] = df.M1 + df.M2 + df.M3 + df.M5 + df.M6 + df.M7 + df.M8 + df.M9
df['num_matches'] = df['M'].str.count('T')
del df['M']

print(df.shape)

(1097231, 443)
(1097231, 444)


In [72]:
############ RESOLUTION

print(df.shape)

df['pixels'] = df['id_33'].str.split('x', expand = True)[0].fillna(0).astype('int') * df['id_33'].str.split('x', expand = True)[1].fillna(0).astype('int') / 100000

print(df.shape)

(1097231, 444)
(1097231, 445)


In [73]:
############ AMOUNT

print(df.shape)

df['TransactionAmt_dollars'] = df['TransactionAmt'].astype('str').str.split('.', expand = True)[0]
df['TransactionAmt_cents']   = df['TransactionAmt'].astype('str').str.split('.', expand = True)[1]

print(df.shape)

(1097231, 445)
(1097231, 447)


In [74]:
############ DATES

print(df.shape)

start_date = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')

df['TransactionDT'] = df['TransactionDT'].fillna(df['TransactionDT'].median())

df['DT']    = df['TransactionDT'].apply(lambda x: (start_date + datetime.timedelta(seconds = x)))
df['DT_H']  = df['DT'].dt.hour.astype('object')
df['DT_DW'] = df['DT'].dt.dayofweek.astype('object')
df['DT_D']  = (df['DT'].dt.year - 2017)*365 + df['DT'].dt.dayofyear.astype('object')
df['DT_W']  = (df['DT'].dt.year - 2017)*52  + df['DT'].dt.weekofyear.astype('object')
df['DT_M']  = (df['DT'].dt.year - 2017)*12  + df['DT'].dt.month.astype('object')

print(df.shape)

(1097231, 447)
(1097231, 453)


In [75]:
############ TRANSACTION AMOUNT

print(df.shape)

# take logs
df['TransactionAmt'] = np.log1p(df['TransactionAmt'])

# new group indicators
df['DT_M_ProductCD']  = df['DT_M'].astype(str) + '_' + df['ProductCD'].astype(str)
df['DT_M_card4']      = df['DT_M'].astype(str) + '_' + df['card4'].astype(str)
df['DT_M_DeviceType'] = df['DT_M'].astype(str) + '_' + df['DeviceType'].astype(str)
df['address']         = df['addr1'].astype(str) + '_' + df['addr2'].astype(str)
df['card']            = df['card1'].astype(str) + '_' + df['card2'].astype(str) + '_' + df['card3'].astype(str) + '_' + df['card4'].astype(str) + '_' + df['card5'].astype(str) + '_' + df['card6'].astype(str)
df['product_type']    = df['TransactionAmt'].astype(str) + '_' + df['ProductCD'].astype(str)

df['card1']  = df['card1'].astype(str)
df['card2']  = df['card2'].astype(str)
df['card3']  = df['card3'].astype(str)
df['card4']  = df['card4'].astype(str)
df['addr1']  = df['addr1'].astype(str)
df['addr2']  = df['addr2'].astype(str)

# aggregate by month or other features
for uid in ['DT_M', 'address', 'card', 'DT_DW', 'DT_D', "DT_H", 
            'ProductCD', 'card1', 'card2', 'card3', 'card4', 'DeviceType', 'browser_type', 'operating_system', 'device_name', 'addr1', 'addr2',
            'DT_M_ProductCD', 'DT_M_card4', 'DT_M_DeviceType']:
    agg_df = aggregate_data(df[['TransactionAmt', uid]], group_var = uid, num_stats = ['mean', 'sum', 'min', 'max', 'std'], var_label = uid)
    agg_df[uid] = agg_df[uid].astype('object')
    df = df.merge(agg_df, how = 'left', on = uid)
    del agg_df
    
# drop group indicators
for uid in ['DT_M_ProductCD', 'DT_M_card4', 'DT_M_DeviceType']:
    del df[uid]
    
# compute difference in means
#for uid in ['DT_M', 'DT_M_ProductCD', 'DT_M_card4', 'DT_M_DeviceType']:
#    df['TransactionAmt'] - df[uid + '_TransactionAmt_mean']

print(df.shape)

(1097231, 453)
- Preparing the dataset...
- Extracted 0 factors and 1 numerics...
- Aggregating numeric features...
- Final dimensions: (12, 6)
- Preparing the dataset...
- Extracted 0 factors and 1 numerics...
- Aggregating numeric features...
- Final dimensions: (608, 6)
- Preparing the dataset...
- Extracted 0 factors and 1 numerics...
- Aggregating numeric features...
- Final dimensions: (19311, 6)
- Preparing the dataset...
- Extracted 0 factors and 1 numerics...
- Aggregating numeric features...
- Final dimensions: (7, 6)
- Preparing the dataset...
- Extracted 0 factors and 1 numerics...
- Aggregating numeric features...
- Final dimensions: (365, 6)
- Preparing the dataset...
- Extracted 0 factors and 1 numerics...
- Aggregating numeric features...
- Final dimensions: (24, 6)
- Preparing the dataset...
- Extracted 0 factors and 1 numerics...
- Aggregating numeric features...
- Final dimensions: (5, 6)
- Preparing the dataset...
- Extracted 0 factors and 1 numerics...
- Aggregatin

In [76]:
# compute difference in means
for uid in ['DT_M', 'DT_M_ProductCD', 'DT_M_card4', 'DT_M_DeviceType']:
    df[uid + '_TransactionAmt_mean_dif1'] = df['TransactionAmt'] - df[uid + '_TransactionAmt_mean']
    df[uid + '_TransactionAmt_mean_dif2'] = np.abs(df[uid + '_TransactionAmt_mean_dif1']) / df[uid + '_TransactionAmt_std']
    
print(df.shape)

(1097231, 564)


In [77]:
############ COUNTS

print(df.shape)

for var in ['card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'address', 'card', 'product_type', 'id_30', 'id_31', 'id_33', 'DeviceInfo']:
    temp = df[var].value_counts().to_dict()
    df[var + '_counts'] = df[var].map(temp)
    
print(df.shape)

(1097231, 564)
(1097231, 579)


In [78]:
############ REMOVE NOISY IDS

print(df.shape)

del df['id_30'], df['id_31'], df['id_33'], df['address'], df['card'], df['product_type'], df['DeviceInfo']

drop_col = ['V300','V309','V111','V124','V106','V125','V315','V134','V102','V123','V316','V113',
              'V136','V305','V110','V299','V289','V286','V318','V304','V116','V284','V293',
              'V137','V295','V301','V104','V311','V115','V109','V119','V321','V114','V133','V122','V319',
              'V105','V112','V118','V117','V121','V108','V135','V320','V303','V297','V120',
              'V1','V14','V41','V65','V88', 'V89', 'V107', 'V68', 'V28', 'V27', 'V29', 'V241','V269',
              'V240', 'V325', 'V138', 'V154', 'V153', 'V330', 'V142', 'V195', 'V302', 'V328', 'V327', 
              'V198', 'V196', 'V155']
for var in drop_col:
    del df[var]
    
print(df.shape)

(1097231, 579)
(1097231, 498)


# EXPORT

In [79]:
# export data
df.to_pickle("../input/data_v8.pkl")
df.shape

(1097231, 498)